# Shared Layer

## Manav Patel - 500967756
## EE8228

### (a) Pretrain BERT on Short Jokes Datast for MLM

In [1]:
# Importing from the src directory
import sys
sys.path.append('./src')

# Import the data loading and preprocessing function
from data import load_and_preprocess, tokenize_and_mask

# Define paths to datasets
train_file_path = './data/shortjokes/train.tsv'
test_file_path = './data/shortjokes/test.tsv'

# Load and preprocess the datasets
train_dataset_mlm = load_and_preprocess(train_file_path, "Training dataset")
test_dataset_mlm = load_and_preprocess(test_file_path, "Testing dataset")

# Display the first 5 rows of the datasets
print("\nFirst 5 rows of the training dataset:")
print(train_dataset_mlm[:5])

print("\nFirst 5 rows of the testing dataset:")
print(test_dataset_mlm[:5])



Successfully loaded Training dataset from ./data/shortjokes/train.tsv
Training dataset contains 173635 humorous and 173850 non-humorous jokes before filtering.
Filtered out 173850 non-humorous jokes from Training dataset.
Sampled 75,000 humorous jokes from Training dataset.
Converted Training dataset to Hugging Face Dataset with 75000 samples.
Successfully loaded Testing dataset from ./data/shortjokes/test.tsv
Sampled 7,500 jokes from Testing dataset.
Converted Testing dataset to Hugging Face Dataset with 7500 samples.

First 5 rows of the training dataset:
{'text': ['god it is time i punished the humans again jesus cool. flood or plague god[watching the apprentice] oh i have something way worse in mind..', 'sorry feminists... why do doctors slap babies butts when they come out to knock the balls off the dumb ones.', 'someone fucked up i got gary glitter in the mail.', 'the guyz l shadowrun returns l razor edge ep  l hes still there hi im karmit and i play video link well that is all i

In [2]:
# Tokenize and mask the training dataset
train_tokenized = tokenize_and_mask(train_dataset_mlm)

# Tokenize and mask the testing dataset
test_tokenized = tokenize_and_mask(test_dataset_mlm)
# Set the format of the datasets to PyTorch tensors
train_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# View a sample from the tokenized training dataset
print("\nSample tokenized and masked data from training dataset:")
print(train_tokenized[0])

# View a sample from the tokenized testing dataset
print("\nSample tokenized and masked data from testing dataset:")
print(test_tokenized[0])


Map:   0%|          | 0/75000 [00:00<?, ? examples/s]

Tokenization and masking completed. Dataset now has features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Tokenization and masking completed. Dataset now has features: {'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

Sample tokenized and masked data from training dataset:
{'input_ids': tensor([  101,  2643,  2009,  2003,  2051,  1045, 14248,  1996,  4286,  2153,
         4441,  4658,   103,  7186,  2030, 11629,  2643,   103,  3666,   103,
        13357,  1033,  2821,  1045,   103,  2242,  2126,  4788,  1999,  2568,
         1012,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,    

In [3]:
from transformers import BertForMaskedLM

# Initialize the BERT model for Masked Language Modeling
model_mlm = BertForMaskedLM.from_pretrained("bert-base-uncased")


2024-11-21 09:48:50.299457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedMod

In [4]:
from training import initialize_trainer, start_training, shared_evaluate_model
# Initialize the Trainer
trainer_mlm, model_mlm = initialize_trainer(
    model=model_mlm,                      # Your initial BERT MLM model instance
    train_dataset=train_tokenized,        # Tokenized training dataset
    eval_dataset=test_tokenized,          # Tokenized evaluation dataset
    output_dir='./models/bert-mlm',       # Output directory for models and checkpoints
    logging_dir='./output/logs',          # Logging directory
    learning_rate=5e-5,                   # Learning rate
    num_train_epochs=4,                   # Number of training epochs
    per_device_train_batch_size=16,       # Training batch size
    per_device_eval_batch_size=32,        # Evaluation batch size
    logging_steps=500,                    # Log metrics every 500 steps
    save_steps=10_000,                    # Save checkpoint every 10,000 steps
    eval_steps=10_000,                    # Evaluate every 10,000 steps
    save_total_limit=2                     # Keep only the 2 most recent checkpoints
)

Found existing model in ./models/bert-mlm. Loading the model...


In [5]:
# Start the training process
train_result_mlm, trained_model_mlm = start_training(trainer_mlm)

if train_result_mlm is None:
    print("Training skipped. Pretrained model loaded.")
else:
    print("Training completed and model saved.")

Training skipped. Pretrained model loaded.
Training skipped. Pretrained model loaded.


In [6]:
from transformers import BertTokenizer
from training import shared_initialize_classification_trainer, shared_start_training
from data import shared_load_and_preprocess_classification_data, shared_tokenize_classification_data


In [7]:
# Path to the classification dataset
shared_data_path = './data/preprocessed_combined_dataset.csv'

# Load and preprocess the classification dataset
shared_dataset, shared_label_encoder = shared_load_and_preprocess_classification_data(shared_data_path)

# Split the dataset into training and evaluation sets
shared_dataset = shared_dataset.train_test_split(test_size=0.2)
shared_train = shared_dataset['train']
shared_test = shared_dataset['test']

print(f"Train dataset size: {len(shared_train)}")
print(f"Eval dataset size: {len(shared_test)}")


Label distribution before sampling:
label
0    210925
1    210730
Name: count, dtype: int64
Balanced dataset created with 150000 samples.
Label distribution after balancing:
label
0    75000
1    75000
Name: count, dtype: int64
Loaded and preprocessed classification data. Number of samples: 150000
Train dataset size: 120000
Eval dataset size: 30000


In [8]:
# Tokenize the training and evaluation datasets
train_tokenized_shared = shared_tokenize_classification_data(shared_train)
eval_tokenized_shared = shared_tokenize_classification_data(shared_test)

print(f"Tokenized train dataset size: {len(train_tokenized_shared)}")
print(f"Tokenized eval dataset size: {len(eval_tokenized_shared)}")


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Tokenization for classification completed.


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Tokenization for classification completed.
Tokenized train dataset size: 120000
Tokenized eval dataset size: 30000


In [9]:
# Initialize the classification trainer
shared_trainer_classification, shared_classification_model = shared_initialize_classification_trainer(
    model_dir='./models/bert-mlm',                   # Directory of the pretrained MLM model
    train_dataset=train_tokenized_shared,            # Tokenized training dataset
    eval_dataset=eval_tokenized_shared,              # Tokenized evaluation dataset
    output_dir='./models/bert-classification',       # Output directory for classification model
    logging_dir='./output/logs_classification',      # Logging directory
    learning_rate=5e-5,                              # Learning rate
    num_train_epochs=4,                              # Number of training epochs
    per_device_train_batch_size=16,                  # Training batch size
    per_device_eval_batch_size=32,                   # Evaluation batch size
    logging_steps=500,                               # Log metrics every 500 steps
    save_steps=10_000,                               # Save checkpoint every 10,000 steps
    eval_steps=10_000,                               # Evaluate every 10,000 steps
    save_total_limit=2,                              # Keep only the 2 most recent checkpoints
    gradient_accumulation_steps=4                    # Simulate larger batch size
)


Found existing classification model in ./models/bert-classification. Loading the model...


In [10]:
# Start the classification training process
shared_train_result, shared_trained_model = shared_start_training(shared_trainer_classification)

if shared_train_result is None:
    print("Training was skipped. Pretrained classification model loaded.")
else:
    print("Classification training completed and model saved.")


Training skipped. Pretrained classification model loaded.
Training was skipped. Pretrained classification model loaded.


In [12]:

import os
import json
import numpy as np

# Convert NumPy arrays and other non-serializable objects in the metrics dictionary to serializable types
def make_json_serializable(data):
    """
    Recursively converts non-serializable objects (e.g., NumPy arrays) in a dictionary to serializable types.
    """
    if isinstance(data, dict):
        return {key: make_json_serializable(value) for key, value in data.items()}
    elif isinstance(data, np.ndarray):
        return data.tolist()  # Convert NumPy arrays to lists
    elif isinstance(data, (np.float32, np.float64)):  # Convert NumPy floats to Python floats
        return float(data)
    elif isinstance(data, (np.int32, np.int64)):  # Convert NumPy integers to Python integers
        return int(data)
    return data  # Leave other types unchanged


# Example: Saving the metrics to ./output/reports
output_dir = "./output/reports"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

# Convert the metrics to a JSON-serializable format
serializable_metrics = make_json_serializable(metrics)

# Filepath to save the metrics
output_file = os.path.join(output_dir, "evaluation_metrics.json")

# Save the metrics to a JSON file
with open(output_file, "w") as f:
    json.dump(serializable_metrics, f, indent=4)

print(f"Metrics successfully saved to {output_file}")

Metrics successfully saved to ./output/reports/evaluation_metrics.json


In [27]:
import sys
from transformers import BertForSequenceClassification
from training import shared_evaluate_model
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

sys.path.append('./output')

shared_data_path2 = './data/humor_nonhumor_dataset_v7.csv'

# Load the CSV file
df2 = pd.read_csv(shared_data_path2)

# Clean the 'text' column
df2['text'] = df2['text'].astype(str).str.strip()

# Check if there are enough samples for both labels
label_counts2 = df2['label'].value_counts()
print(f"Label distribution before sampling:\n{label_counts2}")

# Sample 15,000 rows for each label
balanced_df2 = pd.concat([
    df2[df2['label'] == 0].sample(n=15000, random_state=42),
    df2[df2['label'] == 1].sample(n=15000, random_state=42)
])

# Shuffle the balanced dataset
balanced_df2 = balanced_df2.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Balanced dataset created with {len(balanced_df2)} samples.")
print(f"Label distribution after balancing:\n{balanced_df2['label'].value_counts()}")

# Encode labels
label_encoder2 = LabelEncoder()
balanced_df2['labels'] = label_encoder2.fit_transform(balanced_df2['label'])

# Convert to Hugging Face Dataset
dataset2 = Dataset.from_pandas(balanced_df2[['text', 'labels']])

print(f"Loaded and preprocessed classification data. Number of samples: {len(dataset2)}")

print(dataset2[:10])

# Treat the entire dataset as the test set (no split)
shared_train2 = dataset2.select([])  # Empty training set
shared_test2 = dataset2  # Entire dataset as test set

print(f"Train dataset size: {len(shared_train2)}")  # Will be 0
print(f"Eval dataset size: {len(shared_test2)}")  # Will be the entire dataset

# Tokenizing the evaluation dataset using the existing tokenization function
eval_tokenized_shared2 = shared_tokenize_classification_data(shared_test2)

# Load the pre-trained BERT model
shared_classification_model2 = BertForSequenceClassification.from_pretrained('./models/bert-classification')

print(shared_classification_model2)

# Evaluate the model
metrics2 = shared_evaluate_model(shared_classification_model2, eval_tokenized_shared2, batch_size=32, sample_fraction=0.9)
print("Overall Accuracy:", metrics2['accuracy'])


Label distribution before sampling:
label
1    15000
0    15000
Name: count, dtype: int64
Balanced dataset created with 30000 samples.
Label distribution after balancing:
label
0    15000
1    15000
Name: count, dtype: int64
Loaded and preprocessed classification data. Number of samples: 30000
{'text': ['Shakespeare wrote many famous plays, including Hamlet and Romeo and Juliet.', 'I’m reading a book about anti-gravity. It’s impossible to put down.', 'I don’t trust people who do acupuncture—they’re back stabbers.', "I don't trust stairs. They're always up to something.", 'An apple a day keeps the doctor away.', 'The speed of sound is faster in water than in air.', 'The speed of light is approximately 299,792 kilometers per second.', 'The capital of France is Paris.', 'Water boils at 100 degrees Celsius at sea level.', 'What do you call a pile of cats? A meow-tain.'], 'labels': [0, 1, 1, 1, 0, 0, 0, 0, 0, 1]}
Train dataset size: 0
Eval dataset size: 30000


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Tokenization for classification completed.
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
        

Evaluating: 100%|█████████████████████████████| 844/844 [02:20<00:00,  6.00it/s]


Accuracy: 0.8153
Precision: 0.8651
Recall: 0.8153
F1 Score: 0.8088

Confusion Matrix:
[[ 8519  4987]
 [    0 13494]]
Overall Accuracy: 0.8152962962962963


In [29]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the model and tokenizer
model_path = './models/bert-classification'  # Update with your path to the model
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define 5 jokes and 5 regular sentences
sentences = [
    "Why don't skeletons fight each other? They don't have the guts.",  # Joke 1
    "I told my computer I needed a break, and now it won't stop sending me ads for vacations.",  # Joke 2
    "Why don’t scientists trust atoms? Because they make up everything!",  # Joke 3
    "I asked the librarian if the library had any books on paranoia. She whispered, 'They're right behind you.'",  # Joke 4
    "Parallel lines have so much in common. It’s a shame they’ll never meet.",  # Joke 5
    "The sun sets in the west every evening.",  # Regular sentence 1
    "Mountains are beautiful, but they have no ears.",  # Regular sentence 2
    "The wind howls in the middle of the night.",  # Regular sentence 3
    "She smiled as the clock struck midnight.",  # Regular sentence 4
    "There is a tree in my backyard that grows very tall.",  # Regular sentence 5
]

# Loop through the sentences
for sentence in sentences:
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get prediction (0 or 1)
    prediction = torch.argmax(logits, dim=-1).item()

    # Print result
    if prediction == 0:
        print(f"'{sentence}' is not humorous.")
    else:
        print(f"'{sentence}' is humorous.")


'Why don't skeletons fight each other? They don't have the guts.' is humorous.
'I told my computer I needed a break, and now it won't stop sending me ads for vacations.' is humorous.
'Why don’t scientists trust atoms? Because they make up everything!' is humorous.
'I asked the librarian if the library had any books on paranoia. She whispered, 'They're right behind you.'' is humorous.
'Parallel lines have so much in common. It’s a shame they’ll never meet.' is humorous.
'The sun sets in the west every evening.' is not humorous.
'Mountains are beautiful, but they have no ears.' is humorous.
'The wind howls in the middle of the night.' is not humorous.
'She smiled as the clock struck midnight.' is humorous.
'There is a tree in my backyard that grows very tall.' is humorous.
